## 如何理解"队列"
- 先进者先出，一种受限制的线性表数据结构
- 队列的基本操作：入队（enqueue）和出对（dequeue）
- 循环队列、阻塞队列、并发队列，在底层系统、框架、中间件的开发中，起着关键性作用。比如高性能队列 Disruptor、Linux 环形缓存，都用到了循环并发队列：Java concurrent 并发包利用 ArrayBlockingQueue 来实现公平锁等。

![image](page1.png)

---

## 顺序队列和链式队列
- 数组实现：顺序队列
- 链式队列：链式队列

- 对于栈，只需要一个栈顶指针就可以了
- 对于队列，需要两个指针：一个是 head 指针，指向队头；一个是 tail 指针，指向队尾
- 当 a、b、c、d 依次入队之后，队列中的 head 指针指向下标 0 的位置，tail 指针指向下标为 4 的位置
![image](page2.png)


- 当我们调用两次出队操作后，队列中 head 指针指向下标为 2 的位置，tail 指针仍然指向下标为 4 的位置
![image](page3.png)

### 基于数组实现队列
```
// 用数组实现的队列
public class ArrayQueue {
  // 数组：items，数组大小：n
  private String[] items;
  private int n = 0;
  // head 表示队头下标，tail 表示队尾下标
  private int head = 0;
  private int tail = 0;

  // 申请一个大小为 capacity 的数组
  public ArrayQueue(int capacity) {
    items = new String[capacity];
    n = capacity;
  }

  // 入队
  public boolean enqueue(String item) {
    // 如果 tail == n 表示队列已经满了
    if (tail == n) return false;
    items[tail] = item;
    ++tail;
    return true;
  }

  // 出队
  public String dequeue() {
    // 如果 head == tail 表示队列为空
    if (head == tail) return null;
    // 为了让其他语言的同学看的更加明确，把 -- 操作放到单独一行来写了
    String ret = items[head];
    ++head;
    return ret;
  }
}
```

- 问题：当 tail 移动到最右边，即使数组中仍然有空间，也无法继续添加数据了，如何解决呢？
  - 在入队的时候，如果没有空间，可以集中触发一次数据搬移操作
  - 从代码看，当队列的 tail 指针移动到数组的最右边后，如果有新的数据入列，可以将 head 到 tail 之间的数据，整体搬迁到数组 0 到 (tail - head) 的位置
  - 出队操作的时间复杂度为 O(1)，但入队的时间复杂度为？
```
   // 入队操作，将 item 放入队尾
  public boolean enqueue(String item) {
    // tail == n 表示队列末尾没有空间了
    if (tail == n) {
      // tail ==n && head==0，表示整个队列都占满了
      if (head == 0) return false;
      // 数据搬移
      for (int i = head; i < tail; ++i) {
        items[i-head] = items[i];
      }
      // 搬移完之后重新更新 head 和 tail
      tail -= head;
      head = 0;
    }
    
    items[tail] = item;
    ++tail;
    return true;
  }
```
![image](page4.png)

---


### 基于链表的队列实现方式
- 同样需要两个指针：head 和 tail，分别指向第一个结点和最后一个结点
- 入队时：tail->next= new_node, tail = tail->next;
- 出队时：head = head->next;

![image](page5.png)

## 循环队列
![image](page6.png)

- 图中这个队列的大小为 8，当前 head=4，tail=7。
- 当有一个新的元素 a 入队时，我们放入下标为 7 的位置。
- 但这个时候，我们并不把 tail 更新为 8，而是将其在环中后移一位，到下标为 0 的位置。当再有一个元素 b 入队时，我们将 b 放入下标为 0 的位置，然后 tail 加 1 更新为 1。所以，在 a，b 依次入队之后，循环队列中的元素就变成了下面的样子：
![image](page7.png)

- 要想写出没有 bug 的循环队列的实现代码，最关键的是，确定好队空和队满的判定条件。
- 在用数组实现的非循环队列中，队满的判断条件是 tail == n，队空的判断条件是 head == tail
- 循环队列：队列为空的判断条件仍然是 head == tail。队列满的判断条件就稍微有点复杂了：(tail+1)%n=head
![image](page8.png)

```
public class CircularQueue {
  // 数组：items，数组大小：n
  private String[] items;
  private int n = 0;
  // head 表示队头下标，tail 表示队尾下标
  private int head = 0;
  private int tail = 0;

  // 申请一个大小为 capacity 的数组
  public CircularQueue(int capacity) {
    items = new String[capacity];
    n = capacity;
  }

  // 入队
  public boolean enqueue(String item) {
    // 队列满了
    if ((tail + 1) % n == head) return false;
    items[tail] = item;
    tail = (tail + 1) % n;
    return true;
  }

  // 出队
  public String dequeue() {
    // 如果 head == tail 表示队列为空
    if (head == tail) return null;
    String ret = items[head];
    head = (head + 1) % n;
    return ret;
  }
}
```

---

## 阻塞队列和并发队列
- 阻塞队列其实就是在队列基础上增加了阻塞操作。就是在队列为空的时候，从队头取数据会被阻塞。如果队列已经满了，那么插入数据的操作就会被阻塞，直到队列中有空闲位置后再插入数据，然后再返回
- 上述的定义就是一个“生产者 - 消费者模型”！
![image](page9.png)

- 线程安全的队列我们叫做并发队列。最简单直接的实现方式是直接在 enqueue()、dequeue() 方法上加锁，但是锁粒度大并发度会比较低，同一时刻仅允许一个存或者取操作。实际上，基于数组的循环队列，利用 CAS 原子操作，可以实现非常高效的并发队列。
- 这也是循环队列比链式队列应用更加广泛的原因。
---

## 队列在线程池等有限资源池中的应用
- 线程池底层的数据结构：队列

### 线程池没有空闲线程时，新的任务请求线程资源时，线程池该如何处理？
- 一般有两种策略：一种是非阻塞的，直接拒绝任务请求；另一种是阻塞式的，将请求排队，等到有空闲线程时，取出排队的请求继续处理
- 基于链表实现方式，可以实现一个无界队列，但可能会导致过多请求排队等待，请求处理的响应时间过长
- 基于数组实现的有界队列，队列大小有限，所以线程池中排队的请求超过队列大小时，接下来的请求就会拒绝；对响应时间敏感的系统，更加合适
---

## 思考题
- 除了线程池这种池结构会用到队列排队请求，你还知道有哪些类似的池结构或者场景中会用到队列的排队请求呢？
- 并发队列，关于如何实现无锁并发队列，网上有非常多的讨论。对这个问题，你怎么看呢？